In [1]:
import gzip
import os
import datetime
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook

/home/achang/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess = None

def reset_tf(sess = None, log_device_placement = False):
    if sess:
        sess.close()
    tf.reset_default_graph()
    tf.set_random_seed(0)
    return tf.InteractiveSession(config = tf.ConfigProto(log_device_placement = log_device_placement))

def dump_statistics():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        print('parameters for "%s": %d' % (variable.name, variable_parameters))
        total_parameters += variable_parameters
    print('total parameters: %d' % total_parameters)

In [3]:
class HyperParameters:
    learning_rate = 1e-3
    
    dropout_rate = 0.0
    
    context_size = 850
    question_size = 60
    answers_size = 6
    
    d_hidden = 200
    
    num_attn_layers_contexts = 3
    num_attn_layers_questions = 3
    num_attn_layers_joint = 3

    dataset_batch_size = 48
    dataset_num_parallel_calls = 4
    dataset_prefetch_size = 128
    dataset_shuffle_size = 1000
    
    max_distance_bias = 15
    
    gradient_clip_norm = 5
    
    loss_pos_weight = 100.0

In [4]:
class AttentionModel:
    def __init__(self, session, word_embeddings, hparams):
        self._session = session
        self._word_embeddings = word_embeddings
        self._hparams = hparams
        
    def _parse_example(self, example_proto):
        # parse proto
        parsed = tf.parse_single_example(example_proto, features = {
            'context': tf.VarLenFeature(tf.int64),
            'question': tf.VarLenFeature(tf.int64),
            'answer_starts': tf.VarLenFeature(tf.int64),
            'answer_ends': tf.VarLenFeature(tf.int64), })
        
        # convert to dense tensors
        context = tf.sparse_tensor_to_dense(parsed['context'])
        question = tf.sparse_tensor_to_dense(parsed['question'])
        answer_starts = tf.sparse_tensor_to_dense(parsed['answer_starts'])
        answer_ends = tf.sparse_tensor_to_dense(parsed['answer_ends'])
        
        # pad tensors
        context_len = tf.shape(context)[0]
        question_len = tf.shape(question)[0]
        answers_len = tf.shape(answer_starts)[0]
        zero_vector = self._word_embeddings.shape[0] - 1
        context = tf.pad(
            context,
            [[0, self._hparams.context_size - context_len]],
            constant_values = 0)
        question = tf.pad(
            question,
            [[0, self._hparams.question_size - question_len]],
            constant_values = 0)
        answer_starts = tf.pad(
            answer_starts,
            [[0, self._hparams.answers_size - answers_len]],
            constant_values = -1)
        answer_ends = tf.pad(
            answer_ends,
            [[0, self._hparams.answers_size - answers_len]],
            constant_values = -1)
        
        return (context, question, answer_starts, answer_ends)
    
    def _build_dataset_pipeline(self):
        with tf.variable_scope('dataset'):
            # placeholders
            self._dataset_filenames = tf.placeholder(
                tf.string,
                shape = [None],
                name = 'dataset_filenames')
            self._dataset_limit = tf.placeholder_with_default(
                tf.constant(-1, tf.int64),
                shape = [],
                name = 'dataset_limit')
            self._dataset_shuffle_size = tf.placeholder_with_default(
                tf.constant(self._hparams.dataset_batch_size, tf.int64),
                shape = [],
                name = 'dataset_shuffle_size')
            self._dataset_batch_size = tf.placeholder_with_default(
                tf.constant(self._hparams.dataset_batch_size, tf.int64),
                shape = [],
                name = 'dataset_batch_size')
            self._dataset_prefetch_size = tf.placeholder_with_default(
                tf.constant(self._hparams.dataset_prefetch_size, tf.int64),
                shape = [],
                name = 'dataset_prefetch_size')

            # build dataset
            dataset = tf.data.TFRecordDataset(
                tf.random_shuffle(self._dataset_filenames),
                compression_type='GZIP')
            dataset = dataset.take(self._dataset_limit)
            dataset = dataset.map(
                self._parse_example,
                num_parallel_calls = self._hparams.dataset_num_parallel_calls)
            dataset = dataset.shuffle(self._dataset_shuffle_size)
            dataset = dataset.prefetch(self._dataset_prefetch_size)
            dataset = dataset.batch(self._dataset_batch_size)

            # build iterator
            self._dataset_iterator = dataset.make_initializable_iterator()
            (contexts, questions, answer_starts, answer_ends) = self._dataset_iterator.get_next()
            
            # give key tensors names
            self._contexts = tf.identity(contexts, 'contexts')
            self._questions = tf.identity(questions, 'questions')
            self._answer_starts = tf.identity(answer_starts, 'answer_starts')
            self._answer_ends = tf.identity(answer_ends, 'answer_ends')

            # hint static shapes
            self._contexts.set_shape([None, self._hparams.context_size])
            self._questions.set_shape([None, self._hparams.question_size])
            self._answer_starts.set_shape([None, self._hparams.answers_size])
            self._answer_ends.set_shape([None, self._hparams.answers_size])

            # minibatch size
            self._minibatch_size = tf.shape(self._contexts)[0]
            self._minibatch_size = tf.identity(self._minibatch_size, 'minibatch_size')
            
            # context positions
            p = tf.range(self._hparams.context_size, dtype = tf.int64)
            p = tf.tile(p, [self._minibatch_size])
            p = tf.reshape(
                p,
                [self._minibatch_size, self._hparams.context_size],
                name = 'context_positions')
            self._context_positions = p

            # question positions
            p = tf.range(self._hparams.question_size, dtype = tf.int64)
            p = tf.tile(p, [self._minibatch_size])
            p = tf.reshape(
                p,
                [self._minibatch_size, self._hparams.question_size],
                name = 'question_positions')
            self._question_positions = p
            
    def _attention_layer(self,
                         keys,
                         queries,
                         values,
                         size = None,
                         distance_bias = False,
                         mask_type = None):
        with tf.variable_scope('attention'):
            # default size
            if size is None:
                size = keys.shape[-1].value
            
            # variables
            key_projection = tf.get_variable(
                'key_projection',
                [keys.shape[-1].value, size])
            query_projection = tf.get_variable(
                'query_projection',
                [queries.shape[-1].value, size])
            
            # extract # queries/keys (must be statically known)
            num_queries = queries.shape[-2].value
            num_keys = keys.shape[-2].value
            
            # compute weights
            q = tf.tensordot(queries, query_projection, axes = 1) # [batch_size, num_queries, size]
            q.set_shape([None, queries.shape[-2].value, size])
            k = tf.tensordot(keys, key_projection, axes = 1)      # [batch_size, num_keys, size]
            k.set_shape([None, keys.shape[-2].value, size])
            k = tf.transpose(k, perm = [0, 2, 1])                 # [batch_size, size, num_keys]
            w = tf.matmul(q, k)                                   # [batch_size, num_queries, num_keys]
            w /= np.sqrt(size)
            
            # apply distance bias
            if distance_bias:
                bias = tf.constant(
                    [[-max(float(np.abs(i - j)), self._hparams.max_distance_bias)
                        for j in range(num_keys)]
                        for i in range(num_queries)])
                bias = tf.expand_dims(bias, axis = 0)             # [1, num_queries, num_keys]
                bias *= self._distance_scaling_factor
                w += bias
            
            # apply mask
            if mask_type is not None:
                infinity= 1e25
                if mask_type == 'f':
                    mask = [[-infinity if i <= j else infinity
                        for j in range(num_keys)]
                        for i in range(num_queries)]
                    mask[0][0] = infinity
                    mask = tf.constant(mask)
                elif mask_type == 'b':
                    mask = [[-infinity if i >= j else infinity
                        for j in range(num_keys)]
                        for i in range(num_queries)]
                    mask[-1][-1] = infinity
                    mask = tf.constant(mask)
                elif mask_type == 's':
                    mask = [[-infinity if i == j else infinity
                        for j in range(num_keys)]
                        for i in range(num_queries)]
                    mask = tf.constant(mask)
                mask = tf.expand_dims(mask, axis = 0)             # [1, num_queries, num_keys]
                w = tf.minimum(w, mask)

            # softmax
            w = tf.nn.softmax(w, name = 'weights')
            
            # apply weights
            return tf.matmul(w, values)
        
#     def _attention_layer_self(self, layer):
#         # grab layer size
#         size = layer.shape[-1].value
#        
#         # self-attention
#         attn = self._attention_layer(
#             layer,
#             layer,
#             layer,
#             distance_bias = True,
#             mask_type = 's')
#        
#         return self._fusion_layer([layer, attn], size)
#    
#     def _fusion_layer(self, layers, size):
#         with tf.variable_scope('fusion'):
#             # fuse self-attention layer
#             layer = tf.concat(layers, axis = -1)
#
#             # feed-forward hidden layer
#             layer = tf.layers.dense(
#                 layer,
#                 size * 2,
#                 activation = tf.nn.relu,
#                 name = 'ff_hidden')
#
#             # feed-forward output layer
#             layer = tf.layers.dense(
#                 layer,
#                 size,
#                 name = 'ff_output')
#
#             # batch norm
#             layer = tf.layers.batch_normalization(
#                 layer,
#                 training = self._training)
#
#             # dropout
#             layer = tf.layers.dropout(
#                 layer,
#                 rate = self._hparams.dropout_rate,
#                 training = self._training)
#
#             return layer
#
#     def _layer_norm(self, layer, epsilon = 1e-6):
#         with tf.variable_scope('layer_norm'):
#             size = layer.shape[-1].value
#             scale = tf.get_variable(
#                 'layer_norm_scale',
#                 [size],
#                 initializer = tf.ones_initializer())
#             bias = tf.get_variable(
#                 'layer_norm_bias',
#                 [size],
#                 initializer = tf.zeros_initializer())
#             mean = tf.reduce_mean(
#                 layer,
#                 axis = -1,
#                 keep_dims = True)
#             variance = tf.reduce_mean(
#                 tf.square(layer - mean),
#                 axis = -1,
#                 keep_dims = True)
#             norm_layer = (layer - mean) * tf.rsqrt(variance + epsilon)
#             return norm_layer * scale + bias

    def _build_model(self):
        with tf.variable_scope('model'):
            # placeholders
            self._training = tf.placeholder(tf.bool, name = 'training')

            # attention distance scale
            self._distance_scaling_factor = tf.get_variable(
                'distance_scaling_factor',
                shape = [],
                initializer = tf.constant_initializer([0.3]))
            
            # word embeddings
            word_embeddings = tf.get_variable(
                name = "word_embeddings",
                shape = self._word_embeddings.shape,
                initializer = tf.constant_initializer(self._word_embeddings),
                trainable = False)
            contexts_embedded = tf.nn.embedding_lookup(
                word_embeddings,
                self._contexts)
            questions_embedded = tf.nn.embedding_lookup(
                word_embeddings,
                self._questions)
            
            # position embeddings
            position_embeddings = tf.get_variable(
                'position_embeddings',
                [self._hparams.context_size, self._hparams.d_hidden],
                dtype = tf.float32)
            context_positions_embedded = tf.nn.embedding_lookup(
                position_embeddings,
                self._context_positions)
            question_positions_embedded = tf.nn.embedding_lookup(
                position_embeddings,
                self._question_positions)
            
            # transform contexts
            with tf.variable_scope('contexts'):
                contexts_layer = tf.layers.dense(
                    contexts_embedded,
                    self._hparams.d_hidden,
                    activation = tf.nn.relu)
                contexts_layer += context_positions_embedded
                contexts_layer = tf.layers.batch_normalization(
                    contexts_layer,
                    training = self._training)
            
            # embed questions w/ positions
            with tf.variable_scope('questions'):
                questions_layer = tf.layers.dense(
                    questions_embedded,
                    self._hparams.d_hidden,
                    activation = tf.nn.relu)
                questions_layer += question_positions_embedded
                questions_layer = tf.layers.batch_normalization(
                    questions_layer,
                    training = self._training)
            
            # context self-attention layers
            for i in range(self._hparams.num_attn_layers_contexts):
                with tf.variable_scope('contexts_self_%d' % i):
                    attn = self._attention_layer(
                        contexts_layer,
                        contexts_layer,
                        contexts_layer,
                        distance_bias = True,
                        mask_type = 's')
                    contexts_layer += attn
                    contexts_layer = tf.layers.batch_normalization(
                        contexts_layer,
                        training = self._training)

            # question self-attention layers
            for i in range(self._hparams.num_attn_layers_questions):
                with tf.variable_scope('questions_self_%d' % i):
                    attn = self._attention_layer(
                        questions_layer,
                        questions_layer,
                        questions_layer,
                        distance_bias = True,
                        mask_type = 's')
                    questions_layer += attn
                    questions_layer = tf.layers.batch_normalization(
                        questions_layer,
                        training = self._training)

            # joint attention layer
            with tf.variable_scope('joint'):
                attn = self._attention_layer(
                    queries = contexts_layer,
                    keys = questions_layer,
                    values = questions_layer)
                joint_layer = contexts_layer + attn
                joint_layer = tf.layers.batch_normalization(
                    joint_layer,
                    training = self._training)

            # joint self-attention layers
            for i in range(self._hparams.num_attn_layers_joint):
                with tf.variable_scope('joint_self_%d' % i):
                    attn = self._attention_layer(
                        joint_layer,
                        joint_layer,
                        joint_layer,
                        distance_bias = True,
                        mask_type = 's')
                    joint_layer += attn
                    joint_layer = tf.layers.batch_normalization(
                        joint_layer,
                        training = self._training)

            # output: answer logits
            self._answer_logits = tf.layers.dense(
                joint_layer,
                1,
                name = 'answer_logits')
            self._answer_logits = tf.squeeze(          # [batch_size, context_size]
                self._answer_logits,
                axis = -1,
                name = 'answer_logits')

    def _build_optimizer(self):
        with tf.variable_scope('optimize'):
            # answer mask
            a0 = tf.sequence_mask(
                self._answer_starts[:, 0],
                self._hparams.context_size,
                dtype = tf.int32)
            a1 = tf.sequence_mask(
                self._answer_ends[:, 0] + 1,
                self._hparams.context_size,
                dtype = tf.int32)
            self._answers = tf.identity(a1 - a0, 'answers')

            # individual losses
            losses = tf.nn.weighted_cross_entropy_with_logits(
                targets = tf.cast(self._answers, tf.float32),
                logits = self._answer_logits,
                pos_weight = self._hparams.loss_pos_weight)

            # total loss
            self._total_loss = tf.reduce_sum(losses) / tf.cast(self._hparams.context_size, tf.float32)
            self._total_loss = tf.identity(self._total_loss, 'total_loss')
            
            # mean loss
            self._mean_loss = self._total_loss / tf.cast(self._minibatch_size, tf.float32)
            self._mean_loss = tf.identity(self._mean_loss, 'mean_loss')
            
            # estimated answers
            self._answer_probs = tf.sigmoid(
                self._answer_logits,
                name = 'answer_probs')
            self._answer_estimates = tf.cast(
                self._answer_probs > 0.5,
                tf.int32,
                name = 'answer_estimates')

            # F1
            self._total_true_positives = tf.reduce_sum(
                self._answers * self._answer_estimates,
                name = 'total_true_positives')
            self._total_false_positives = tf.reduce_sum(
                (1 - self._answers) * self._answer_estimates,
                name = 'total_false_positives')
            self._total_false_negatives = tf.reduce_sum(
                self._answers * (1 - self._answer_estimates),
                name = 'total_false_negatives')
            
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                self._global_step = tf.Variable(0, name = 'global_step', trainable = False)
                self._optimizer = tf.train.AdamOptimizer(learning_rate = self._hparams.learning_rate)
                
                # gradient clipping
                gradients, variables = zip(*self._optimizer.compute_gradients(self._mean_loss))
                gradients, _ = tf.clip_by_global_norm(
                    gradients, 
                    self._hparams.gradient_clip_norm)
                
                self._train_op = self._optimizer.apply_gradients(
                    zip(gradients, variables),
                    global_step = self._global_step)
                
    def process(self,
                dataset_filenames,
                dataset_limit = -1,
                header = 'results',
                train = False,
                log_file = None):
        # initialize dataset to files
        self._session.run(self._dataset_iterator.initializer, feed_dict={
            self._dataset_filenames: dataset_filenames,
            self._dataset_limit: dataset_limit })

        cum_loss = 0
        cum_num_examples = 0
        cum_tps = 0
        cum_fps = 0
        cum_fns = 0
        
        # start progress
        start = datetime.datetime.now()
        progress = tqdm_notebook(leave = False, desc = header)

        while True:
            # process a minibatch
            try:
                (_,
                 curr_total_loss,
                 curr_tps,
                 curr_fps,
                 curr_fns,
                 curr_minibatch_size) = self._session.run(
                    (self._train_op if train else (),
                     self._total_loss,
                     self._total_true_positives,
                     self._total_false_positives,
                     self._total_false_negatives,
                     self._minibatch_size),
                    feed_dict = { self._training: train })
            except tf.errors.OutOfRangeError:
                break

            # update loss stats
            cum_loss += curr_total_loss
            cum_tps += curr_tps
            cum_fps += curr_fps
            cum_fns += curr_fns
            cum_num_examples += curr_minibatch_size
            
            # update progress
            progress.update(curr_minibatch_size)
            progress.set_postfix(loss = cum_loss / cum_num_examples)

        # end progress
        progress.close()
        finish = datetime.datetime.now()
        
        # precision
        precision = 0
        if cum_tps + cum_fps > 0:
            precision = cum_tps / (cum_tps + cum_fps)
            
        # recall
        recall = 0
        if cum_tps + cum_fns > 0:
            recall = cum_tps / (cum_tps + cum_fns)
            
        # F1
        F1 = 0
        if precision + recall > 0:
            F1 = 2 * precision * recall / (precision + recall)
        
        # print/log output
        message = '%s: time=%s, step=%d, loss=%g, precision=%g, recall=%g, F1=%g' % (
            header,
            finish - start,
            tf.train.global_step(sess, self._global_step),
            cum_loss / cum_num_examples,
            precision,
            recall,
            F1)
        print(message)
        if log_file:
            print(message, file=log_file)
            log_file.flush()

In [5]:
with gzip.open('../../data/SQuAD/data_1.vocab.embeddings.npy.gz', 'rb') as f:
    word_embeddings = np.load(f)

In [6]:
def list_files(path):
    return sorted([os.path.join(path, file) for file in os.listdir(path)])

train_set = list_files('../../data/SQuAD/data_1.train')
dev_set = list_files('../../data/SQuAD/data_1.dev')

In [7]:
sess = reset_tf(sess)

model = AttentionModel(sess, word_embeddings, HyperParameters())
model._build_dataset_pipeline()
model._build_model()
model._build_optimizer()
dump_statistics()

parameters for "model/distance_scaling_factor:0": 1
parameters for "model/position_embeddings:0": 170000
parameters for "model/contexts/dense/kernel:0": 60000
parameters for "model/contexts/dense/bias:0": 200
parameters for "model/contexts/batch_normalization/gamma:0": 200
parameters for "model/contexts/batch_normalization/beta:0": 200
parameters for "model/questions/dense/kernel:0": 60000
parameters for "model/questions/dense/bias:0": 200
parameters for "model/questions/batch_normalization/gamma:0": 200
parameters for "model/questions/batch_normalization/beta:0": 200
parameters for "model/contexts_self_0/attention/key_projection:0": 40000
parameters for "model/contexts_self_0/attention/query_projection:0": 40000
parameters for "model/contexts_self_0/batch_normalization/gamma:0": 200
parameters for "model/contexts_self_0/batch_normalization/beta:0": 200
parameters for "model/contexts_self_1/attention/key_projection:0": 40000
parameters for "model/contexts_self_1/attention/query_project

In [8]:
sess.run(tf.global_variables_initializer())

In [ ]:
with open('../../logs/SQuAD/model_attention_2.1.log', 'wt') as f:
    for i in range(5):
        model.process(
            train_set,
            header = 'train_%d' % i,
            train = True,
            log_file = f)
        model.process(
            dev_set,
            header = 'dev_%d' % i,
            train = False,
            log_file = f)

HBox(children=(IntProgress(value=1, bar_style='info', description='train_0', max=1), HTML(value='')))

In [34]:
sess.run(
    model._dataset_iterator.initializer,
    feed_dict = {
        model._dataset_filenames: train_set[:1],
        model._dataset_limit: 100 })

In [35]:
contexts, questions, answers, answer_estimates = sess.run(
    [model._contexts,
     model._questions,
     model._answers,
     model._answer_estimates],
    feed_dict = { model._training: False })

In [36]:
answers[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [44]:
answer_estimates[9]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,